In [5]:
from pynq import Overlay
import pynq.lib.dma
from pynq import DefaultIP
from pynq import allocate
from pynq import MMIO

import numpy as np
import pandas as pd
from scipy import stats, signal

import time
overlay = Overlay("./cg4002/design_1_wrapper.bit")

In [6]:
# overlay?

In [7]:
NN_IP_BASE_ADDRESS = 0x80000000
ADDRESS_RANGE = 0x10000
ADDRESS_OFFSET = 0x0

mmio = MMIO(NN_IP_BASE_ADDRESS, ADDRESS_RANGE)
ap_start = 0x1
ap_auto_reset = 0x80
mmio.write(ADDRESS_OFFSET, ap_start|ap_auto_reset)

In [9]:
def compute_mean(data):
    return np.mean(data)

def compute_variance(data):
    return np.var(data)

def compute_median_absolute_deviation(data):
    return stats.median_absolute_deviation(data)

def compute_root_mean_square(data):
    def compose(*fs):
        def wrapped(x):
            for f in fs[::-1]:
                x = f(x)
            return x
        return wrapped
    rms = compose(np.sqrt, np.mean, np.square)
    return rms(data)

def compute_interquartile_range(data):
    return stats.iqr(data)

def compute_percentile_75(data):
    return np.percentile(data, 75)

def compute_kurtosis(data):
    return stats.kurtosis(data)

def compute_min_max(data):
    return np.max(data) - np.min(data)

def compute_signal_magnitude_area(data):
    return np.sum(data) / len(data)

def compute_zero_crossing_rate(data):
    return ((data[:-1] * data[1:]) < 0).sum()

def compute_spectral_centroid(data):
    spectrum = np.abs(np.fft.rfft(data))
    normalized_spectrum = spectrum / np.sum(spectrum)  
    normalized_frequencies = np.linspace(0, 1, len(spectrum))
    spectral_centroid = np.sum(normalized_frequencies * normalized_spectrum)
    return spectral_centroid

def compute_spectral_entropy(data):
    freqs, power_density = signal.welch(data)
    return stats.entropy(power_density)

def compute_spectral_energy(data):
    freqs, power_density = signal.welch(data)
    return np.sum(np.square(power_density))

def compute_principle_frequency(data):
    freqs, power_density = signal.welch(data)
    return freqs[np.argmax(np.square(power_density))]

def extract_raw_data_features_per_row(f_n):
    f1_mean = compute_mean(f_n)
    f1_var = compute_variance(f_n)
    f1_mad = compute_median_absolute_deviation(f_n)
    f1_rms = compute_root_mean_square(f_n)
    f1_iqr = compute_interquartile_range(f_n)
    f1_per75 = compute_percentile_75(f_n)
    f1_kurtosis = compute_kurtosis(f_n)
    f1_min_max = compute_min_max(f_n)
    f1_sma = compute_signal_magnitude_area(f_n)
    f1_zcr = compute_zero_crossing_rate(f_n)
    f1_sc = compute_spectral_centroid(f_n)
    f1_entropy = compute_spectral_entropy(f_n)
    f1_energy = compute_spectral_energy(f_n)
    f1_pfreq = compute_principle_frequency(f_n)
    return f1_mean, f1_var, f1_mad, f1_rms, f1_iqr, f1_per75, f1_kurtosis, f1_min_max, f1_sma, f1_zcr, f1_sc, f1_entropy, f1_energy, f1_pfreq

def extract_raw_data_features(X):
    new_features = []
    rows = X.shape[0]
    cols = X.shape[1]
    c = 0
    for row in range(rows):
        features = []
        for col in range(cols):
            f_n = np.array(X[row][col])
            feature = extract_raw_data_features_per_row(f_n)
            features.append(feature)
        new_features.append(features)

    return new_features

# cols = ["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z"]
# final_data = extract_raw_data_features(np.array(pdata[cols]))

In [10]:
def create_featureslist(final_data):
    new_data = []
    for row in range(0,np.array(final_data).shape[0]):
        row_features = []
        for col in range(0,np.array(final_data).shape[1]):
            feature_list = np.array(final_data)[row][col]
            for feature in feature_list:
                row_features.append(feature)
        new_data.append(row_features)
        print("Row: " + str(row) + "done")

    feature_name_list = ["_mean", "_var", "_mad", "_rms", "_iqr", "_per75", "_kurtosis", "_min_max", "_sma", "_zcr", "_sc", "_entropy", "_energy", "_pfreq"]
    cols = []
    for things in ["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z"]:
        for feature in feature_name_list:
            cols.append(things+feature)

    data = pd.DataFrame(data=np.array(new_data), columns=cols)
    data = data.dropna()
    return(data)

In [54]:
def combine_rows(data):
    combined_raw = []
    new_data = []
#     new_data = pd.DataFrame(columns=['acceleration_x', 'acceleration_y', 'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z'])
    for col in range(6):
        for row in range(len(data)):
            combined_raw.append(data[row][col])
        new_data.append(combined_raw)
        combined_raw = []
    return new_data

In [56]:
#define a function which accepts raw data parameter.
#raw data would be a queue the form 600 * ['W',acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z]
raw_data = []
pkt1 = ('W',0.1,0.2,0.3,0.4,0.5,0.6)
pkt2 = ('W',0.8,0.9,1.0,1.1,1.2,1.3)
pkt3 = ('W',1.1,1.2,1.3,1.4,1.5,1.6)
raw_data.append(pkt1)
raw_data.append(pkt2)
raw_data.append(pkt3)

def process(raw_data):
    data = pd.DataFrame(columns=['acceleration_x', 'acceleration_y', 'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z'])
    for i in range(len(raw_data)):
        list_row = [raw_data[i][1],raw_data[i][2],raw_data[i][3],raw_data[i][4],raw_data[i][5],raw_data[i][6]]
        data.loc[len(data)] = list_row
    new_data = combine_rows(data.values.tolist())
#     data = pd.DataFrame(data=new_data, columns = ["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z"])
    final_data = extract_raw_data_features((data))
#     final_data = create_featureslist(final_data)
#     data = np.array(data)
    return final_data

# def process(raw_data):
#     print("Hello. I have received the raw data." + "This is the first packed." + raw_data[0])
#     return 1

process(raw_data)      
    
#return lower case string // if not an action, return none.

KeyError: 0

In [57]:
import pandas as pd
def load_file(filepath):
  dataframe = pd.read_csv(filepath)
  return dataframe

data = load_file("./cg4002/data.csv")
data.head()
data = data.dropna()
data

,Unnamed: 0,acc_x_mean,acc_x_var,acc_x_mad,acc_x_rms,acc_x_iqr,acc_x_per75,acc_x_kurtosis,acc_x_min_max,acc_x_sma,...,gyr_z_per75,gyr_z_kurtosis,gyr_z_min_max,gyr_z_sma,gyr_z_zcr,gyr_z_sc,gyr_z_entropy,gyr_z_energy,gyr_z_pfreq,prediction
0,0,0.46044,0.022009,0.243814,0.483749,0.265925,0.594350,-1.331043,0.4072,0.46044,...,1.385475,-1.556672,5.5086,-0.50710,5.0,0.500121,1.287280,1132.655381,0.2,0
1,1,0.42823,0.018363,0.197408,0.449159,0.248225,0.547525,-1.431807,0.3940,0.42823,...,1.065550,-1.320788,4.8366,-0.06403,4.0,0.442374,0.949082,401.792665,0.2,0
2,2,0.47561,0.029778,0.250337,0.505948,0.314975,0.618625,-1.597028,0.4492,0.47561,...,1.449350,-1.204176,6.6787,-0.12131,3.0,0.425174,1.093174,1295.570519,0.2,0
3,3,0.46633,0.012490,0.069312,0.479535,0.125875,0.546150,-0.433022,0.3397,0.46633,...,1.665625,-1.415387,5.3821,0.13797,3.0,0.507171,0.977303,252.500279,0.2,0
4,4,0.46177,0.020286,0.138771,0.483236,0.248825,0.587775,-1.262027,0.4197,0.46177,...,1.616950,-1.386171,4.7962,0.15487,3.0,0.463101,1.049546,372.152507,0.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8865,8865,0.34745,0.014946,0.102892,0.368330,0.120900,0.403525,-0.403102,0.4289,0.34745,...,1.370200,-1.598039,3.8245,-0.04372,3.0,0.456671,1.013194,305.669051,0.2,0
8866,8866,0.36058,0.014079,0.113938,0.379601,0.181650,0.438775,-1.066518,0.3406,0.36058,...,0.712650,-0.604442,5.7829,-0.01267,3.0,0.570799,1.233250,216.762039,0.2,0
8867,8867,0.36241,0.021008,0.145295,0.390319,0.162150,0.438300,-0.663331,0.5080,0.36241,...,0.468825,-0.926718,4.2000,-0.27887,3.0,0.385936,1.184971,319.745040,0.2,0
8868,8868,0.38028,0.030471,0.129060,0.418430,0.144600,0.466350,0.704367,0.6459,0.38028,...,1.688125,-1.300901,5.8583,0.17293,3.0,0.472225,1.365184,414.433217,0.2,0


In [58]:
x = data[data.columns[:84]].astype('float64').to_numpy()
x

array([[0.00000000e+00, 4.60440000e-01, 2.20085004e-02, ...,
        5.00120754e-01, 1.28728011e+00, 1.13265538e+03],
       [1.00000000e+00, 4.28230000e-01, 1.83632041e-02, ...,
        4.42374241e-01, 9.49081506e-01, 4.01792665e+02],
       [2.00000000e+00, 4.75610000e-01, 2.97784209e-02, ...,
        4.25173861e-01, 1.09317399e+00, 1.29557052e+03],
       ...,
       [8.86700000e+03, 3.62410000e-01, 2.10077509e-02, ...,
        3.85935584e-01, 1.18497054e+00, 3.19745040e+02],
       [8.86800000e+03, 3.80280000e-01, 3.04706096e-02, ...,
        4.72225247e-01, 1.36518421e+00, 4.14433217e+02],
       [8.86900000e+03, 3.42220000e-01, 1.21723856e-02, ...,
        4.88595707e-01, 1.20188975e+00, 1.25037805e+02]])

In [59]:
y = data["prediction"].to_numpy()
x = data[data.columns[1:85]].to_numpy()

In [60]:
x = np.array([0.75] * 84)
y = np.array([0])
y.shape

(1,)

In [ ]:
correct_pred = 0
#Setting a threshold for onset detection
# THRESHOLD = 0.01
# threshold_sum = 0
dma = overlay.axi_dma_0        
# take in 10 datapoints, average out, compare to threshold
in_buffer = allocate(shape=x.shape, dtype=np.float32)
out_buffer = allocate(shape=(1,), dtype=np.float32)
for i in range(x.shape[0]):
#     inputs = x[n]
#     for i in range(84):
    in_buffer[i] = x[i];
#         threshold_sum += inputs[i];
#         print(threshold_sum)
# print(in_buffer)
# print(out_buffer)
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
print("aaa")
dma.recvchannel.wait()

out_buffer
# if y[i] == np.argmax(out_buffer):
#     correct_pred += 1;

# accuracy = 100 * correct_pred / len(x)
# print('Accuracy: {}'. format(accuracy))
# print('Process time: ' + str(time.time() - start) + "s")
